<h1>Fast Embeddings</h1>

AlloyDB provides a very useful [embedding() function](https://cloud.google.com/alloydb/docs/ai/work-with-embeddings#embedding-generation) that creates embeddings directly in the database. However, this function does not perform well when generating large batches of embeddings.

This notebook walks you through generating [Vertex AI embeddings](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings-api) for the AlloyDB database used by the [GenWealth Demo App](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/sample-apps/genwealth). It uses [Beam/Dataflow](https://github.com/apache/beam/blob/master/examples/notebooks/beam-ml/data_preprocessing/vertex_ai_text_embeddings.ipynb) to significantly speed up the process of generating large batches of embeddings and storing them in AlloyDB vs using the native embedding() function.

<h2>Setup</h2>

1. Install and import necessary packages.

In [151]:
! pip install SQLAlchemy==2.0.29 --quiet
! pip install google-cloud-alloydb-connector[pg8000]==1.0.0 --quiet
! pip install apache_beam[gcp]>=2.53.0 --quiet


from google.cloud.alloydb.connector import Connector
import sqlalchemy
from tabulate import tabulate
import apache_beam as beam
from apache_beam.ml.transforms.base import MLTransform
from apache_beam.ml.transforms.embeddings.vertex_ai import VertexAITextEmbeddings

import os, shutil
import tempfile
import json

2. Define variables to match your local environment.
>This step assumes you have a secret stored in Secret Manager called `alloydb-secret`. You can use an alternate method to define your password if desired.

In [153]:
# GCP vars
region = "us-central1"
project_id = "YOUR-PROJECT-ID"

# AlloyDB Vars
cluster = "alloydb-cluster"
instance = "alloydb-instance"
database = "ragdemos"
user = "postgres"
password = !gcloud secrets versions access latest --secret="alloydb-secret"
password = str(password[0])

# Embedding vars
text_embedding_model_name = 'textembedding-gecko@003'

3. Setup the database connection to AlloyDB. This connector and pool will be used later in the notebook.

In [154]:
# Define connector and pool
connector = Connector()

def getconn():
    conn = connector.connect(
        f"projects/{project_id}/locations/{region}/clusters/{cluster}/instances/{instance}",
        "pg8000",
        user=user,
        password=password,
        db=database,
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)


4. Retrieve the text data from AlloyDB that you want to embed.

In [155]:
# Store output in array of serializable dictionaries
result_dicts = []

# Define database query to get primary key plus text data to embed
# Ensure you retrieve the id key to uniquely identify the row you are embedding
sql = f"""
    SELECT id, overview, analysis FROM embedding_test;
    """

# Run the query
print(f"Running SQL query: {sql}")
with pool.connect() as db_conn:
    # query database
    result = db_conn.execute(sqlalchemy.text(sql))

    for row in result:
      result_dicts.append(dict(row._mapping))

    db_conn.commit()


Running SQL query: 
    SELECT id, overview, analysis FROM embedding_test;
    


5. Define helper function to updates embeddings in AlloyDB.

In [156]:
# Helper function to update embeddings in AlloyDB
def update_embeddings(_transformed_pcoll):

  # Define the update query
  sql = f"""
    UPDATE embedding_test SET
      overview_embedding = '{_transformed_pcoll['overview']}',
      analysis_embedding = '{_transformed_pcoll['analysis']}'
    WHERE id = {_transformed_pcoll['id']};
    """

  # Run the query
  with pool.connect() as db_conn:
      # query database
      result = db_conn.execute(sqlalchemy.text(sql))
      db_conn.commit()

In [150]:
# Remove the temp file if it exists (throws an error otherwise)
if os.path.exists(artifact_location):
  shutil.rmtree(artifact_location)

# Create a temp file for beam output
artifact_location = tempfile.mkdtemp(prefix='vertex_ai')

# Define the columns you want to embeding (overview and analysis in this case)
embedding_transform = VertexAITextEmbeddings(
    model_name=text_embedding_model_name, columns=['overview', 'analysis'], project=project_id)

# Define the beam pipeline
with beam.Pipeline() as pipeline:
  data_pcoll = (
      pipeline
      | "CreateData" >> beam.Create(result_dicts))
  transformed_pcoll = (
      data_pcoll
      | "MLTransform" >> MLTransform(write_artifact_location=artifact_location).with_transform(embedding_transform))

  # Update the embeddings in AlloyDB
  transformed_pcoll | beam.Map(update_embeddings)
